Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

# 04. Train in a remote Linux VM
* Create Workspace
* Create `train.py` file
* Create and Attach a Remote VM (eg. DSVM) as compute resource.
* Upoad data files into default datastore
* Configure & execute a run in a few different ways
    - Use system-built conda
    - Use existing Python environment
    - Use Docker 
* Find the best model in the run

## Prerequisites
Make sure you go through the [configuration notebook](../../../configuration.ipynb) first if you haven't.

In [ ]:
# Check core SDK version number
import azureml.core

print("SDK version:", azureml.core.VERSION)

## Initialize Workspace

Initialize a workspace object from persisted configuration.

In [ ]:
from azureml.core import Workspace

ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep='\n')

## Create Experiment

**Experiment** is a logical container in an Azure ML Workspace. It hosts run records which can include run metrics and output artifacts from your experiments.

In [ ]:
experiment_name = 'train-on-remote-vm'

from azureml.core import Experiment
exp = Experiment(workspace=ws, name=experiment_name)

Let's also create a local folder to hold the training script.

In [ ]:
import os
script_folder = './vm-run'
os.makedirs(script_folder, exist_ok=True)

## Upload data files into datastore
Every workspace comes with a default datastore (and you can register more) which is backed by the Azure blob storage account associated with the workspace. We can use it to transfer data from local to the cloud, and access it from the compute target.

In [ ]:
# get the default datastore
ds = ws.get_default_datastore()
print(ds.name, ds.datastore_type, ds.account_name, ds.container_name)

Load diabetes data from `scikit-learn` and save it as 2 local files.

In [ ]:
from sklearn.datasets import load_diabetes
import numpy as np

training_data = load_diabetes()
np.save(file='./features.npy', arr=training_data['data'])
np.save(file='./labels.npy', arr=training_data['target'])

Now let's upload the 2 files into the default datastore under a path named `diabetes`:

In [ ]:
ds.upload_files(['./features.npy', './labels.npy'], target_path='diabetes', overwrite=True)

## View `train.py`

For convenience, we created a training script for you. It is printed below as a text, but you can also run `%pfile ./train.py` in a cell to show the file. Please pay special attention on how we are loading the features and labels from files in the `data_folder` path, which is passed in as an argument of the training script (shown later).

In [ ]:
# copy train.py into the script folder
import shutil
shutil.copy('./train.py', os.path.join(script_folder, 'train.py'))

with open(os.path.join(script_folder, './train.py'), 'r') as training_script:
    print(training_script.read())

## Create and Attach a DSVM as a compute target

**Note**: To streamline the compute that Azure Machine Learning creates, we are making updates to support creating only single to multi-node `AmlCompute`. The DSVM can be created using the below single line command and then attached(like any VM) using the sample code below. Also note, that we only support Linux VMs for remote execution from AML and the commands below will spin a Linux VM only.

```shell
# create a DSVM in your resource group
# note you need to be at least a contributor to the resource group in order to execute this command successfully
(myenv) $ az vm create --resource-group <resource_group_name> --name <some_vm_name> --image microsoft-dsvm:linux-data-science-vm-ubuntu:linuxdsvmubuntu:latest --admin-username <username> --admin-password <password> --generate-ssh-keys --authentication-type password
```

**Note**: You can also use [this url](https://portal.azure.com/#create/microsoft-dsvm.linux-data-science-vm-ubuntulinuxdsvmubuntu) to create the VM using the Azure Portal

**Note**: By default SSH runs on port 22 and you don't need to specify it. But if for security reasons you switch to a different port (such as 5022), you can specify the port number in the provisioning configuration object.

In [ ]:
from azureml.core.compute import ComputeTarget, RemoteCompute
from azureml.core.compute_target import ComputeTargetException

username = os.getenv('AZUREML_DSVM_USERNAME', default='<my_username>')
address = os.getenv('AZUREML_DSVM_ADDRESS', default='<ip_address_or_fqdn>')

compute_target_name = 'cpudsvm'
# if you want to connect using SSH key instead of username/password you can provide parameters private_key_file and private_key_passphrase 
try:
    attached_dsvm_compute = RemoteCompute(workspace=ws, name=compute_target_name)
    print('found existing:', attached_dsvm_compute.name)
except ComputeTargetException:
    attach_config = RemoteCompute.attach_configuration(address=address,
                                                        ssh_port=22,
                                                        username=username,
                                                        private_key_file='./.ssh/id_rsa')
    attached_dsvm_compute = ComputeTarget.attach(workspace=ws,
                                                 name=compute_target_name,
                                                 attach_config=attach_config)
    attached_dsvm_compute.wait_for_completion(show_output=True)

## Configure & Run
First let's create a `DataReferenceConfiguration` object to inform the system what data folder to download to the copmute target.

In [ ]:
from azureml.core.runconfig import DataReferenceConfiguration
dr = DataReferenceConfiguration(datastore_name=ds.name, 
                   path_on_datastore='diabetes', 
                   mode='download', # download files from datastore to compute target
                   overwrite=True)

Now we can try a few different ways to run the training script in the VM.

### Conda run
You can ask the system to build a conda environment based on your dependency specification, and submit your script to run there. Once the environment is built, and if you don't change your dependencies, it will be reused in subsequent runs.

In [ ]:
from azureml.core.runconfig import RunConfiguration
from azureml.core.conda_dependencies import CondaDependencies

# create a new RunConfig object
conda_run_config = RunConfiguration(framework="python")

# Set compute target to the Linux DSVM
conda_run_config.target = attached_dsvm_compute.name

# set the data reference of the run configuration
conda_run_config.data_references = {ds.name: dr}

# specify CondaDependencies obj
conda_run_config.environment.python.conda_dependencies = CondaDependencies.create(conda_packages=['scikit-learn'])

In [ ]:
from azureml.core import ScriptRunConfig

src = ScriptRunConfig(source_directory=script_folder, 
                      script='train.py', 
                      run_config=conda_run_config, 
                      # pass the datastore reference as a parameter to the training script
                      arguments=['--data-folder', str(ds.as_download())] 
                     ) 
run = exp.submit(config=src)

In [ ]:
run.wait_for_completion(show_output=True)

Show the run object. You can navigate to the Azure portal to see detailed information about the run.

In [ ]:
run

### Native VM run
You can also configure to use an exiting Python environment in the VM to execute the script without asking the system to create a conda environment for you.

In [ ]:
# create a new RunConfig object
vm_run_config = RunConfiguration(framework="python")

# Set compute target to the Linux DSVM
vm_run_config.target = attached_dsvm_compute.name

# set the data reference of the run coonfiguration
conda_run_config.data_references = {ds.name: dr}

# Let system know that you will configure the Python environment yourself.
vm_run_config.environment.python.user_managed_dependencies = True

The below run will likely fail because `train.py` needs dependency `azureml`, `scikit-learn` and others, which are not found in that Python environment. 

In [ ]:
src = ScriptRunConfig(source_directory=script_folder, 
                      script='train.py', 
                      run_config=vm_run_config,
                      # pass the datastore reference as a parameter to the training script
                      arguments=['--data-folder', str(ds.as_download())])
run = exp.submit(config=src)
run.wait_for_completion(show_output=True)

You can choose to SSH into the VM and install Azure ML SDK, and any other missing dependencies, in that Python environment. For demonstration purposes, we simply are going to use another script `train2.py` that doesn't have azureml dependencies, and submit it instead.

In [ ]:
# copy train2.py into the script folder
shutil.copy('./train2.py', os.path.join(script_folder, 'train2.py'))

with open(os.path.join(script_folder, './train2.py'), 'r') as training_script:
    print(training_script.read())

Now let's try again. And this time it should work fine.

In [ ]:
src = ScriptRunConfig(source_directory=script_folder, 
                      script='train2.py', 
                      run_config=vm_run_config)
run = exp.submit(config=src)
run.wait_for_completion(show_output=True)

Note even in this case you get a run record with some basic statistics.

In [ ]:
run

### Configure a Docker run with new conda environment on the VM
You can execute in a Docker container in the VM. If you choose this option, the system will pull down a base Docker image, build a new conda environment in it if you ask for (you can also skip this if you are using a customer Docker image when a preconfigured Python environment), start a container, and run your script in there. This image is also uploaded into your ACR (Azure Container Registry) assoicated with your workspace, an reused if your dependencies don't change in the subsequent runs.

In [ ]:
# Load the "cpu-dsvm.runconfig" file (created by the above attach operation) in memory
docker_run_config = RunConfiguration(framework="python")

# Set compute target to the Linux DSVM
docker_run_config.target = attached_dsvm_compute.name

# Use Docker in the remote VM
docker_run_config.environment.docker.enabled = True

# Use CPU base image from DockerHub
docker_run_config.environment.docker.base_image = azureml.core.runconfig.DEFAULT_CPU_IMAGE
print('Base Docker image is:', docker_run_config.environment.docker.base_image)

# set the data reference of the run coonfiguration
docker_run_config.data_references = {ds.name: dr}

# specify CondaDependencies obj
docker_run_config.environment.python.conda_dependencies = CondaDependencies.create(conda_packages=['scikit-learn'])

### Submit the Experiment
Submit script to run in the Docker image in the remote VM. If you run this for the first time, the system will download the base image, layer in packages specified in the `conda_dependencies.yml` file on top of the base image, create a container and then execute the script in the container.

In [ ]:
src = ScriptRunConfig(source_directory=script_folder, 
                      script='train.py', 
                      run_config=docker_run_config,
                      # pass the datastore reference as a parameter to the training script
                      arguments=['--data-folder', str(ds.as_download())])
run = exp.submit(config=src)

In [ ]:
run.wait_for_completion(show_output=True)

### View run history details

In [ ]:
run

### Find the best model

Now we have tried various execution modes, we can find the best model from the last run.

In [ ]:
# get all metris logged in the run
run.get_metrics()
metrics = run.get_metrics()

In [ ]:
# find the index where MSE is the smallest
indices = list(range(0, len(metrics['mse'])))
min_mse_index = min(indices, key=lambda x: metrics['mse'][x])

print('When alpha is {1:0.2f}, we have min MSE {0:0.2f}.'.format(
    metrics['mse'][min_mse_index], 
    metrics['alpha'][min_mse_index]
))

## Clean up compute resource

Use ```detach()``` to detach an existing DSVM from Workspace without deleting it. Use ```delete()``` if you created a new ```DsvmCompute``` and want to delete it.

In [ ]:
# dsvm_compute.detach()
# dsvm_compute.delete()